In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 2.3MB 8.0MB/s 
     |████████████████████████████████| 1.2MB 34.8MB/s 
     |████████████████████████████████| 3.3MB 34.9MB/s 
     |████████████████████████████████| 901kB 36.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=a9d15dda07fd7fb8bca2640688dc7b481f318884cdca854d39811ea905634675
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [ ]:
"""
This script trains sentence transformers with a triplet loss function.
As corpus, we use the wikipedia sections dataset that was describd by Dor et al., 2018, Learning Thematic Similarity Metric Using Triplet Networks.
"""

from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from datetime import datetime
from zipfile import ZipFile

import csv
import logging
import os

In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

logging.info("Read test dataset")
test_sts_samples = []
with open("/content/sample_data/sentence_pair_unique.csv", 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn)
    for row in reader:
        score = float(row['avg']) / 4.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row['sent1_text'], row['sent2_text']], label=score)
        test_sts_samples.append(inp_example)

In [ ]:
model_name = 'stsb-bert-base'
output_path = "output/training-wikipedia-sections-"+model_name+"-"+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model = SentenceTransformer(model_name)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_sts_samples, name='sts-test')
test_evaluator(model, output_path=output_path)

FileNotFoundError: ignored

In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df_test = pd.read_csv("/content/sample_data/sentence_pair_unique.csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df_test.shape[0]))

# Display 5 random rows from the data.
df_test.sample(1)

# Get the lists of sentences and their labels.
sent1 = df_test.sent1_text.values
sent2 = df_test.sent2_text.values
sent1 = sent1.tolist()
sent2 = sent2.tolist()

sent1_embeddings = model.encode(sent1)
sent1_embeddings.shape

sent2_embeddings = model.encode(sent2)
sent2_embeddings.shape

from sklearn.metrics.pairwise import cosine_similarity
import scipy
import pandas as pd
import numpy as np


cos_list = []
cos2_list = []

for j in range(len(sent1_embeddings)):
  sent1 = sent1_embeddings[j]
  sent2 = sent2_embeddings[j]
  cos = 1-scipy.spatial.distance.cdist(sent1.reshape(1, -1), sent2.reshape(1, -1), "cosine")[0][0]
  cos2 = cosine_similarity(sent1.reshape(1, -1), sent2.reshape(1, -1))[0][0]
  cos_list.append(cos)
  cos2_list.append(cos2)

assert(len(cos_list)==len(df_test))
assert(len(cos2_list)==len(df_test))

df_test["model_similarity"] = cos_list
df_test["model_similarity2"] = cos2_list

df_test.to_csv("/content/sample_data/sentence_pair_sbert.csv",
                    index=None)

Number of training sentences: 1,635



# Pairs

In [ ]:
### Create a torch.DataLoader that passes training batch instances to our model

model_name = 'stsb-bert-base'
train_batch_size = 8
output_path = "output/training-amazon-pair-"+model_name+"-"+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
num_epochs = 1
# n_train = 1000 
# n_train = 3000
n_train = 8000
trip_type = "firstsentence"

model = SentenceTransformer(model_name)

2021-05-13 02:55:59 - Load pretrained SentenceTransformer: stsb-bert-base
2021-05-13 02:55:59 - Did not find folder stsb-bert-base
2021-05-13 02:55:59 - Search model on server: http://sbert.net/models/stsb-bert-base.zip
2021-05-13 02:55:59 - Load SentenceTransformer from folder: /root/.cache/torch/sentence_transformers/sbert.net_models_stsb-bert-base


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_stsb-bert-base/0_BERT were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2021-05-13 02:56:01 - Use pytorch device: cuda


In [ ]:
logger.info("Read Amazon Pair train dataset")
train_examples = []
with open("/content/sample_data/amazon_pair_train_10000_" + trip_type + ".csv", encoding="utf-8") as fIn:
    reader = csv.DictReader(fIn)
    for row in reader:
     # print(row)
      train_examples.append(InputExample(texts=[row['sent1_text'], row['sent2_text']], label=int(row['label'])))

2021-05-13 02:56:01 - Read Amazon Pair train dataset


In [ ]:
test_examples = train_examples[0:1000]
dev_examples = train_examples[1000:2000]
train_examples = train_examples[2000:(2000+n_train)]

In [ ]:
print(len(test_examples))
print(len(dev_examples))
print(len(train_examples))

1000
1000
7997


In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=2)

2021-05-13 02:56:01 - Softmax loss: #Vectors concatenated: 3


In [ ]:
logger.info("Read Amazon Pair dev dataset")

evaluator = BinaryClassificationEvaluator.from_input_examples(dev_examples, name='amazon-dev_sbertft_' + str(n_train) + "_" + trip_type + "_run1")

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

2021-05-13 02:56:01 - Read Amazon Pair dev dataset


In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=output_path)

2021-05-13 03:02:09 - Binary Accuracy Evaluation of the model on amazon-dev_sbertft_8000_firstsentence_run1 dataset in epoch 0 after 1000 steps:
2021-05-13 03:02:16 - Accuracy with Cosine-Similarity:           81.60	(Threshold: 0.2154)
2021-05-13 03:02:16 - F1 with Cosine-Similarity:                 82.36	(Threshold: 0.0995)
2021-05-13 03:02:16 - Precision with Cosine-Similarity:          78.67
2021-05-13 03:02:16 - Recall with Cosine-Similarity:             86.42
2021-05-13 03:02:16 - Average Precision with Cosine-Similarity:  89.29

2021-05-13 03:02:16 - Accuracy with Manhatten-Distance:           81.20	(Threshold: 423.8982)
2021-05-13 03:02:16 - F1 with Manhatten-Distance:                 81.89	(Threshold: 451.5259)
2021-05-13 03:02:16 - Precision with Manhatten-Distance:          80.19
2021-05-13 03:02:16 - Recall with Manhatten-Distance:             83.66
2021-05-13 03:02:16 - Average Precision with Manhatten-Distance:  89.09

2021-05-13 03:02:16 - Accuracy with Euclidean-Distance

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

logger.info("Read test examples")
model = SentenceTransformer(output_path)
test_evaluator = BinaryClassificationEvaluator.from_input_examples(dev_examples, name='amazon-test_sbertft_' + str(n_train) + "_" + trip_type + "_run1")
test_evaluator(model, output_path=output_path)

2021-05-13 03:02:23 - Read test examples
2021-05-13 03:02:23 - Load pretrained SentenceTransformer: output/training-amazon-pair-stsb-bert-base-2021-05-13_02-55-59
2021-05-13 03:02:23 - Load SentenceTransformer from folder: output/training-amazon-pair-stsb-bert-base-2021-05-13_02-55-59
2021-05-13 03:02:25 - Use pytorch device: cuda
2021-05-13 03:02:25 - Binary Accuracy Evaluation of the model on amazon-test_sbertft_8000_firstsentence_run1 dataset:
2021-05-13 03:02:31 - Accuracy with Cosine-Similarity:           81.60	(Threshold: 0.2154)
2021-05-13 03:02:31 - F1 with Cosine-Similarity:                 82.36	(Threshold: 0.0995)
2021-05-13 03:02:31 - Precision with Cosine-Similarity:          78.67
2021-05-13 03:02:31 - Recall with Cosine-Similarity:             86.42
2021-05-13 03:02:31 - Average Precision with Cosine-Similarity:  89.29

2021-05-13 03:02:31 - Accuracy with Manhatten-Distance:           81.20	(Threshold: 423.8982)
2021-05-13 03:02:31 - F1 with Manhatten-Distance:          

0.8928819590196063

In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df_test = pd.read_csv("/content/sample_data/sentence_pair_unique.csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df_test.shape[0]))

# Display 5 random rows from the data.
df_test.sample(1)

# Get the lists of sentences and their labels.
sent1 = df_test.sent1_text.values
sent2 = df_test.sent2_text.values
sent1 = sent1.tolist()
sent2 = sent2.tolist()

sent1_embeddings = model.encode(sent1)
sent1_embeddings.shape

sent2_embeddings = model.encode(sent2)
sent2_embeddings.shape

from sklearn.metrics.pairwise import cosine_similarity
import scipy
import pandas as pd
import numpy as np


cos_list = []
cos2_list = []

for j in range(len(sent1_embeddings)):
  sent1 = sent1_embeddings[j]
  sent2 = sent2_embeddings[j]
  cos = 1-scipy.spatial.distance.cdist(sent1.reshape(1, -1), sent2.reshape(1, -1), "cosine")[0][0]
  cos2 = cosine_similarity(sent1.reshape(1, -1), sent2.reshape(1, -1))[0][0]
  cos_list.append(cos)
  cos2_list.append(cos2)

assert(len(cos_list)==len(df_test))
assert(len(cos2_list)==len(df_test))

df_test["model_similarity"] = cos_list
df_test["model_similarity2"] = cos2_list

df_test.to_csv("/content/sample_data/sentence_pair_sbert_amazon_"+str(n_train)+"_" + trip_type + "_pair_run1.csv",
                    index=None)

Number of training sentences: 1,635



## Triplets

In [ ]:
### Create a torch.DataLoader that passes training batch instances to our model

model_name = 'stsb-bert-base'
train_batch_size = 8
output_path = "output/training-amazon-triplets-"+model_name+"-"+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
num_epochs = 1
# n_train = 1000 
# n_train = 3000
n_train = 8000
trip_margin = 5.0
trip_type = "randomsentence"

model = SentenceTransformer(model_name)

2021-05-13 01:04:07 - Load pretrained SentenceTransformer: stsb-bert-base
2021-05-13 01:04:07 - Did not find folder stsb-bert-base
2021-05-13 01:04:07 - Search model on server: http://sbert.net/models/stsb-bert-base.zip
2021-05-13 01:04:07 - Load SentenceTransformer from folder: /root/.cache/torch/sentence_transformers/sbert.net_models_stsb-bert-base


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_stsb-bert-base/0_BERT were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2021-05-13 01:04:08 - Use pytorch device: cuda


In [ ]:
logger.info("Read Triplet train dataset")
train_examples = []
with open("/content/sample_data/amazon_triplets_train_10000_" + trip_type + ".csv", encoding="utf-8") as fIn:
    reader = csv.DictReader(fIn)
    for row in reader:
        train_examples.append(InputExample(texts=[row['anchor_text'], row['same_text'], row['diff_text']], label=0))

2021-05-13 01:04:08 - Read Triplet train dataset


In [ ]:
len(train_examples)

9996

In [ ]:
test_examples = train_examples[0:1000]
dev_examples = train_examples[1000:2000]
train_examples = train_examples[2000:(2000+n_train)]

In [ ]:
from enum import Enum
from sentence_transformers.losses.TripletLoss import TripletDistanceMetric

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.TripletLoss(model=model, triplet_margin= trip_margin)
# distance_metric=TripletDistanceMetric.COSINE

In [ ]:
len(train_dataloader)

1000

In [ ]:
logger.info("Read Amazon Triplet dev dataset")

evaluator = TripletEvaluator.from_input_examples(dev_examples, name='amazon-dev_sbertft_' + str(n_train) + "_" + trip_type + "_margin" + str(int(trip_margin))+ "_run1")

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data


2021-05-13 01:04:09 - Read Amazon Triplet dev dataset


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=200,
          warmup_steps=warmup_steps,
          output_path=output_path)

2021-05-13 01:04:59 - TripletEvaluator: Evaluating the model on amazon-dev_sbertft_8000_randomsentence_margin5_run1 dataset in epoch 0 after 200 steps:
2021-05-13 01:05:03 - Accuracy Cosine Distance:   	79.60
2021-05-13 01:05:03 - Accuracy Manhatten Distance:	80.00
2021-05-13 01:05:03 - Accuracy Euclidean Distance:	80.80

2021-05-13 01:05:03 - Save model to output/training-amazon-triplets-stsb-bert-base-2021-05-13_01-04-07
2021-05-13 01:05:52 - TripletEvaluator: Evaluating the model on amazon-dev_sbertft_8000_randomsentence_margin5_run1 dataset in epoch 0 after 400 steps:
2021-05-13 01:05:56 - Accuracy Cosine Distance:   	82.40
2021-05-13 01:05:56 - Accuracy Manhatten Distance:	82.10
2021-05-13 01:05:56 - Accuracy Euclidean Distance:	82.20

2021-05-13 01:05:56 - Save model to output/training-amazon-triplets-stsb-bert-base-2021-05-13_01-04-07
2021-05-13 01:06:45 - TripletEvaluator: Evaluating the model on amazon-dev_sbertft_8000_randomsentence_margin5_run1 dataset in epoch 0 after 600 s

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on Amazon test dataset
#
##############################################################################

logger.info("Read test examples")
model = SentenceTransformer(output_path)
test_evaluator = TripletEvaluator.from_input_examples(test_examples, name='amazon-test_sbertft_' + str(n_train) + "_" + trip_type + "_margin" + str(int(trip_margin))+ "_run1")
test_evaluator(model, output_path=output_path)

2021-05-13 01:08:42 - Read test examples
2021-05-13 01:08:42 - Load pretrained SentenceTransformer: output/training-amazon-triplets-stsb-bert-base-2021-05-13_01-04-07
2021-05-13 01:08:42 - Load SentenceTransformer from folder: output/training-amazon-triplets-stsb-bert-base-2021-05-13_01-04-07
2021-05-13 01:08:43 - Use pytorch device: cuda
2021-05-13 01:08:43 - TripletEvaluator: Evaluating the model on amazon-test_sbertft_8000_randomsentence_margin5_run1 dataset:
2021-05-13 01:08:48 - Accuracy Cosine Distance:   	80.90
2021-05-13 01:08:48 - Accuracy Manhatten Distance:	80.40
2021-05-13 01:08:48 - Accuracy Euclidean Distance:	80.70



0.809

In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df_test = pd.read_csv("/content/sample_data/sentence_pair_unique.csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df_test.shape[0]))

# Display 5 random rows from the data.
df_test.sample(1)

# Get the lists of sentences and their labels.
sent1 = df_test.sent1_text.values
sent2 = df_test.sent2_text.values
sent1 = sent1.tolist()
sent2 = sent2.tolist()

sent1_embeddings = model.encode(sent1)
sent1_embeddings.shape

sent2_embeddings = model.encode(sent2)
sent2_embeddings.shape

from sklearn.metrics.pairwise import cosine_similarity
import scipy
import pandas as pd
import numpy as np


cos_list = []
cos2_list = []

for j in range(len(sent1_embeddings)):
  sent1 = sent1_embeddings[j]
  sent2 = sent2_embeddings[j]
  cos = 1-scipy.spatial.distance.cdist(sent1.reshape(1, -1), sent2.reshape(1, -1), "cosine")[0][0]
  cos2 = cosine_similarity(sent1.reshape(1, -1), sent2.reshape(1, -1))[0][0]
  cos_list.append(cos)
  cos2_list.append(cos2)

assert(len(cos_list)==len(df_test))
assert(len(cos2_list)==len(df_test))

df_test["model_similarity"] = cos_list
df_test["model_similarity2"] = cos2_list

df_test.to_csv("/content/sample_data/sentence_pair_sbert_amazon_"+str(n_train)+"_" + trip_type + "_triplet_margin" + str(int(trip_margin)) + "_run1.csv",
                    index=None)

Number of training sentences: 1,635



In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on Amazon test dataset
#
##############################################################################

logger.info("Read test examples")
model = SentenceTransformer(model_name)
test_evaluator = TripletEvaluator.from_input_examples(test_examples, name='amazon-test_sbert')
test_evaluator(model, output_path=output_path)

2021-05-12 09:18:49 - Read test examples
2021-05-12 09:18:49 - Load pretrained SentenceTransformer: stsb-bert-base
2021-05-12 09:18:49 - Did not find folder stsb-bert-base
2021-05-12 09:18:49 - Search model on server: http://sbert.net/models/stsb-bert-base.zip
2021-05-12 09:18:49 - Load SentenceTransformer from folder: /root/.cache/torch/sentence_transformers/sbert.net_models_stsb-bert-base
2021-05-12 09:18:52 - Use pytorch device: cuda
2021-05-12 09:18:52 - TripletEvaluator: Evaluating the model on amazon-test_sbert dataset:
2021-05-12 09:19:12 - Accuracy Cosine Distance:   	72.50
2021-05-12 09:19:12 - Accuracy Manhatten Distance:	74.00
2021-05-12 09:19:12 - Accuracy Euclidean Distance:	73.80



0.74